In [1]:
import csv
from typing import List, Tuple, Dict
from dataclasses import dataclass
import random
import numpy as np
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

@dataclass
class Part:
    location: str
    length: int
    height: int
    quantity: int

@dataclass
class Placement:
    part: Part
    x: int
    y: int
    rotated: bool

class Sheet:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width
        self.placements: List[Placement] = []
        self.remaining_space = [(0, 0, length, width)]

    def add_part(self, part: Part, x: int, y: int, rotated: bool):
        actual_length = part.height if rotated else part.length
        actual_height = part.length if rotated else part.height
        self.placements.append(Placement(part, x, y, rotated))
        
        new_remaining = []
        for space in self.remaining_space:
            sx, sy, sl, sh = space
            if x < sx + sl and y < sy + sh:
                if y > sy:
                    new_remaining.append((sx, sy, sl, y - sy))
                if y + actual_height < sy + sh:
                    new_remaining.append((sx, y + actual_height, sl, sy + sh - (y + actual_height)))
                if x > sx:
                    new_remaining.append((sx, max(sy, y), x - sx, min(sh, actual_height)))
                if x + actual_length < sx + sl:
                    new_remaining.append((x + actual_length, max(sy, y), sx + sl - (x + actual_length), min(sh, actual_height)))
            else:
                new_remaining.append(space)
        self.remaining_space = new_remaining

class HybridOptimizer:
    def __init__(self, parts: List[Part], stock_sizes: List[Tuple[int, int]]):
        self.parts = parts
        self.stock_sizes = stock_sizes

    def genetic_placement(self, population_size=50, generations=20):
        population = self._initialize_population(population_size)
        
        for _ in range(generations):
            # Evaluate fitness
            fitness_scores = [self._calculate_fitness(solution) for solution in population]
            
            # Selection
            selected = self._tournament_selection(population, fitness_scores)
            
            # Crossover
            offspring = self._crossover(selected)
            
            # Mutation
            offspring = self._mutate(offspring)
            
            # Replace population
            population = offspring + selected
        
        # Return best solution
        best_solution = max(population, key=self._calculate_fitness)
        return best_solution

    def _initialize_population(self, population_size):
        population = []
        for _ in range(population_size):
            # Create a random arrangement of parts
            solution = self.parts.copy()
            random.shuffle(solution)
            population.append(solution)
        return population

    def _calculate_fitness(self, solution):
        sheets = self._place_parts(solution)
        # Fitness based on material utilization and minimum sheets used
        utilization = sum(
            sum(p.part.length * p.part.height for p in sheet.placements) / (sheet.length * sheet.width)
            for sheet in sheets
        )
        return utilization / len(sheets)

    def _tournament_selection(self, population, fitness_scores, tournament_size=5):
        selected = []
        for _ in range(len(population) // 2):
            # Select best from a random subset
            tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
            winner = max(tournament, key=lambda x: x[1])[0]
            selected.append(winner)
        return selected

    def _crossover(self, selected):
        offspring = []
        while len(offspring) < len(selected):
            parent1, parent2 = random.sample(selected, 2)
            crossover_point = random.randint(1, len(parent1) - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            offspring.append(child)
        return offspring

    def _mutate(self, population, mutation_rate=0.1):
        for solution in population:
            if random.random() < mutation_rate:
                # Swap two random parts
                i, j = random.sample(range(len(solution)), 2)
                solution[i], solution[j] = solution[j], solution[i]
        return population

    def _place_parts(self, parts_order):
        sheets = []
        remaining_parts = parts_order.copy()
        
        while remaining_parts:
            best_sheet = None
            best_utilization = 0
            
            for length, width in self.stock_sizes:
                sheet = Sheet(length, width)
                for part in remaining_parts:
                    for _ in range(part.quantity):
                        x, y, rotated = self._heuristic_best_fit(sheet, part)
                        if x != -1:
                            sheet.add_part(part, x, y, rotated)
                
                # Calculate sheet utilization
                utilization = sum(p.part.length * p.part.height for p in sheet.placements) / (length * width)
                if utilization > best_utilization:
                    best_utilization = utilization
                    best_sheet = sheet
            
            if best_sheet:
                sheets.append(best_sheet)
                # Remove placed parts
                for placement in best_sheet.placements:
                    part = next(p for p in remaining_parts if p.location == placement.part.location)
                    part.quantity -= 1
                    if part.quantity == 0:
                        remaining_parts.remove(part)
            else:
                break
        
        return sheets

    def _heuristic_best_fit(self, sheet: Sheet, part: Part) -> Tuple[int, int, bool]:
        best_fit = None
        min_waste = float('inf')
        
        for space in sheet.remaining_space:
            x, y, w, h = space
            for rotated in (False, True):
                pl, ph = (part.height, part.length) if rotated else (part.length, part.height)
                if pl <= w and ph <= h:
                    waste = w * h - pl * ph
                    if waste < min_waste:
                        min_waste = waste
                        best_fit = (x, y, rotated)
        
        return best_fit if best_fit else (-1, -1, False)

    def optimize(self):
        # Combine genetic and heuristic approaches
        genetic_solution = self.genetic_placement()
        sheets = self._place_parts(genetic_solution)
        return sheets

def load_glass_data(filepath: str) -> List[Part]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [Part(row['location'], int(row['glass_length']), int(row['glass_height']), int(row['glass_qty'])) for row in reader]

def load_stock_sizes(filepath: str) -> List[Tuple[int, int]]:
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        return [(int(row['length']), int(row['width'])) for row in reader]

def create_visualization_pdf(sheets: List[Sheet], filename: str = 'cutting_optimization.pdf'):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    for i, sheet in enumerate(sheets, 1):
        # Page title
        c.setFont("Helvetica-Bold", 12)
        c.drawString(inch, height - inch, f"Sheet {i}: {sheet.length}x{sheet.width}")

        # Draw sheet outline
        c.rect(inch, height - 4*inch, sheet.length/10, sheet.width/10)

        # Draw placed parts
        c.setFont("Helvetica", 8)
        for placement in sheet.placements:
            x = inch + placement.x / 10
            y = height - 4*inch + placement.y / 10
            w = placement.part.length / 10
            h = placement.part.height / 10
            
            # Rotate if needed
            if placement.rotated:
                w, h = h, w
            
            # Draw rectangle for part
            c.rect(x, y, w, h)
            
            # Add part details
            c.drawString(x, y-0.2*inch, f"{placement.part.location}")

        c.showPage()

    c.save()

def visualize_utilization(sheets: List[Sheet]):
    utilizations = [
        sum(p.part.length * p.part.height for p in sheet.placements) / (sheet.length * sheet.width) * 100
        for sheet in sheets
    ]
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(1, len(sheets) + 1), utilizations)
    plt.title('Sheet Utilization Percentages')
    plt.xlabel('Sheet Number')
    plt.ylabel('Utilization (%)')
    plt.tight_layout()
    plt.savefig('sheet_utilization.png')
    plt.close()

def print_results(sheets: List[Sheet], total_part_area: float):
    total_sheet_area = sum(sheet.length * sheet.width for sheet in sheets) / 1_000_000
    used_area_percentage = (total_part_area / total_sheet_area) * 100
    wastage_percentage = 100 - used_area_percentage

    print(f"\nTotal sheets used: {len(sheets)}")
    print(f"Total part area: {round(total_part_area, 2)} sqm")
    print(f"Total sheet area used: {round(total_sheet_area, 2)} sqm")
    print(f"Used area percentage: {used_area_percentage:.2f}%")
    print(f"Wastage percentage: {wastage_percentage:.2f}%")

    print("\nOptimized Layout:")
    for i, sheet in enumerate(sheets, 1):
        print(f"Sheet {i}: {sheet.length}x{sheet.width}")
        for placement in sheet.placements:
            orientation = "rotated" if placement.rotated else "normal"
            print(f"  {placement.part.location} ({placement.part.length}x{placement.part.height}) at position ({placement.x},{placement.y}) ({orientation})")

def main():
    glass_data_file = 'data/glass_data.csv'
    stock_sizes_file = 'data/glass_sheet_size.csv'

    # Load data
    parts = load_glass_data(glass_data_file)
    stock_sizes = load_stock_sizes(stock_sizes_file)

    # Create optimizer
    optimizer = HybridOptimizer(parts, stock_sizes)

    # Optimize cutting
    optimized_layout = optimizer.optimize()

    # Calculate total part area
    total_part_area = sum(
        placement.part.length * placement.part.height / 1_000_000
        for sheet in optimized_layout 
        for placement in sheet.placements
    )

    # Print results
    print_results(optimized_layout, total_part_area)

    # Create visualizations
    create_visualization_pdf(optimized_layout)
    visualize_utilization(optimized_layout)

if __name__ == "__main__":
    main()

ZeroDivisionError: division by zero